In [1]:
import cppyy
from ctypes import c_ulonglong, c_ulong, c_longlong, c_uint, c_uint64
import ctypes
from array import array

In [2]:
cppyy.load_reflection_info('libBoinkDict.so')

In [3]:
def _ns_boink_pythonizor(klass, name):
    print(name)
    
    if name.startswith('FileProcessor'):
        def chunked(self, filename):
            parser = cppyy.gbl.ParserFactory[boink.parsing.FastxReader].build(filename)
            while True:
                state = self.advance(parser)
                yield self.n_reads(), state
                if state.end:
                    break
        klass.chunked = chunked
        
    if 'interval_state' in name:
        klass.__str__ = lambda self: '<interval_state fine={0} medium={1} coarse={2} end={3}'.format(self.fine, self.medium, self.coarse, self.end)


In [4]:
cppyy.py.add_pythonization(_ns_boink_pythonizor, 'boink')

In [5]:
from cppyy.gbl import boink, std

In [6]:
G = std.make_shared[boink.dBG[boink.storage.SparseppSetStorage, boink.hashing.RollingHashShifter]](31)

dBG<boink::storage::SparseppSetStorage,boink::hashing::RollingHashShifter>


In [6]:
asm = boink.CompactorMixin[boink.dBG[boink.storage.SparseppSetStorage, boink.hashing.RollingHashShifter]](G)

In [6]:
parser = cppyy.gbl.ParserFactory[boink.parsing.FastxReader].build('/work/draff/results/simulated/ecoli.0._100_180_1.fq')

In [7]:
parser

<cppyy.gbl.boink.parsing.ReadParser<boink::parsing::FastxReader> object at 0x55fd88db87a0 held by shared_ptr<boink::parsing::ReadParser<boink::parsing::FastxReader> > at 0x55fd8e1c3760>

In [7]:
proc = boink.FileConsumer[boink.dBG[boink.storage.SparseppSetStorage, boink.hashing.RollingHashShifter]](G)

FileProcessor<boink::FileConsumer<boink::dBG<boink::storage::SparseppSetStorage,boink::hashing::RollingHashShifter>,boink::parsing::FastxReader>,boink::parsing::FastxReader>
FileConsumer<boink::dBG<boink::storage::SparseppSetStorage,boink::hashing::RollingHashShifter>,boink::parsing::FastxReader>


In [8]:
for n_reads, state in proc.chunked('/work/draff/results/simulated/ecoli.0._100_180_1.fq'):
    print(n_reads, state)

interval_state
10000 <interval_state fine=1 medium=0 coarse=0 end=0
20000 <interval_state fine=1 medium=0 coarse=0 end=0
30000 <interval_state fine=1 medium=0 coarse=0 end=0
40000 <interval_state fine=1 medium=0 coarse=0 end=0
50000 <interval_state fine=1 medium=0 coarse=0 end=0
60000 <interval_state fine=1 medium=0 coarse=0 end=0
70000 <interval_state fine=1 medium=0 coarse=0 end=0
80000 <interval_state fine=1 medium=0 coarse=0 end=0
90000 <interval_state fine=1 medium=0 coarse=0 end=0
100000 <interval_state fine=1 medium=0 coarse=0 end=0
110000 <interval_state fine=1 medium=0 coarse=0 end=0
120000 <interval_state fine=1 medium=0 coarse=0 end=0
130000 <interval_state fine=1 medium=0 coarse=0 end=0
140000 <interval_state fine=1 medium=0 coarse=0 end=0
150000 <interval_state fine=1 medium=0 coarse=0 end=0
160000 <interval_state fine=1 medium=0 coarse=0 end=0
170000 <interval_state fine=1 medium=0 coarse=0 end=0
180000 <interval_state fine=1 medium=0 coarse=0 end=0
190000 <interval_state

In [11]:
while True:
    state = proc.advance(parser)
    if state.fine:
        print(proc.n_reads())
    if state.end:
        break

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000


In [15]:
G.n_unique()

9946822

In [21]:
boink.SourmashSignatureProcessor

<class cppyy.gbl.boink.SourmashSignatureProcessor at 0x557f0c487cc8>

In [8]:
cppyy.cppdef('''
void assign_ref(unsigned int src, unsigned int& dst) {
    dst = src;
}
''')

True

In [9]:
src = c_uint(32)
dst = c_uint(0)

In [10]:
print(src)

c_uint(32)


In [11]:
cppyy.gbl.assign_ref(42, dst)

In [12]:
print(dst)

c_uint(42)


In [17]:
start = s[:7]
path = boink.Path()
end_hash = c_ulonglong(32)
mask = std.set[boink.hashing.hash_t]()
asm.set_cursor(start)

16928099823363697171

In [18]:
asm.compactify_right(path, end_hash, mask)
print(asm.to_string(path))

TypeError: void boink::CompactorMixin<boink::dBG<boink::storage::SparseppSetStorage,boink::hashing::RollingHashShifter> >::compactify_right(deque<char>& path, unsigned long& end_hash, set<unsigned long>& mask) =>
    TypeError: could not convert argument 2 (could not convert argument to buffer or nullptr)

In [56]:
path = boink.Path()

In [44]:
shifter = boink.hashing.RollingHashShifter(7)

In [58]:
for i in range(5):
    path.clear()
    shifter.get_cursor(path)
    print(asm.to_string(path))

GGATTCG
GGATTCG
GGATTCG
GGATTCG
GGATTCG


In [45]:
for i in range(len(s) - 9 + 1):
    kmer = s[i:i+9]
    shifter.set_cursor(kmer) 
    print(shifter.get(), shifter.hash(kmer))
    #print(kmer, int(asm.degree_left()), int(asm.degree_right()))

16928099823363697171 16928099823363697171
17774349600126154739 17774349600126154739
5432234656955854598 5432234656955854598
15864785342952959926 15864785342952959926
807854437867792248 807854437867792248
503930369671869761 503930369671869761
14846359574385085156 14846359574385085156
6910144570189067695 6910144570189067695
8953154543898151736 8953154543898151736
15784469927269168548 15784469927269168548
12000143714024755448 12000143714024755448
10045950146635431439 10045950146635431439
3219486363132302806 3219486363132302806
11146315393402538954 11146315393402538954
17774349600126154739 17774349600126154739
5432234656955854598 5432234656955854598
15864785342952959926 15864785342952959926
807854437867792248 807854437867792248
503930369671869761 503930369671869761
14846359574385085156 14846359574385085156


In [19]:
int.from_bytes(asm.degree_left(), byteorder='big')

TypeError: cannot convert 'str' object to bytes

In [21]:
type(asm.degree_left())

str

In [12]:
it = boink.hashing.KmerIterator[boink.hashing.RollingHashShifter](s, 7)

In [13]:
kmer = it.first()
print(kmer)
while not it.done():
    kmer = it.next()
    print(kmer)

16928099823363697171
17774349600126154739
5432234656955854598
15864785342952959926
807854437867792248
503930369671869761
14846359574385085156
6910144570189067695
8953154543898151736
15784469927269168548
12000143714024755448
10045950146635431439
3219486363132302806
11146315393402538954
17774349600126154739
5432234656955854598
15864785342952959926
807854437867792248
503930369671869761
14846359574385085156
6910144570189067695
1873201609706990527


In [42]:
r = G.neighbors('CTGGGAT')

In [43]:
r.first.back().symbol

'G'

In [44]:
new = cppyy.gbl.std.set[boink.hashing.hash_t]()
G.insert_sequence(s + 'GGTGTGGGGGG', new)

0

In [48]:
G.shared_from_this()

<cppyy.gbl.boink.dBG<boink::storage::SparseppSetStorage,boink::hashing::RollingHashShifter> object at 0x55e8a6232490 held by shared_ptr<boink::dBG<boink::storage::SparseppSetStorage,boink::hashing::RollingHashShifter> > at 0x55e8a493dd90>

In [49]:
shifter = boink.hashing.RollingHashShifter('AAAAAAA', 7)

In [26]:
shifter.update_right('C')

17896415738016705498

In [27]:
shifter.get()

17896415738016705498

In [23]:
storages = ['BitStorage', 'ByteStorage', 'NibbleStorage', 'QFStorage', 'SparseppSetStorage']
for storage in storages:
    print('//', storage)
    print('template class boink::dBG<boink::storage::{},\n'
          '                          boink::hashing::RollingHashShifter>;'.format(storage))
    print('template class std::enable_shared_from_this<boink::dBG<boink::storage::{},\n'
          '                                            boink::hashing::RollingHashShifter>>;'.format(storage))
    print('template class boink::AssemblerMixin<boink::dBG<boink::storage::{},\n'
          '                                                 boink::hashing::RollingHashShifter>>;'.format(storage))
    print('template class boink::CompactorMixin<boink::dBG<boink::storage::{},\n'
          '                                                 boink::hashing::RollingHashShifter>>;'.format(storage))
    print('template class boink::DecisionNodeProcessor<boink::dBG<boink::storage::{},\n'
          '                                                       boink::hashing::RollingHashShifter>>;'.format(storage))
    print('template class boink::StreamingCompactorProcessor<boink::dBG<boink::storage::{},\n'
          '                                                             boink::hashing::RollingHashShifter>>;'.format(storage))
    print('template class boink::FileConsumer<boink::dBG<boink::storage::{},\n'
          '                                              boink::hashing::RollingHashShifter>>;'.format(storage))
    print()

// BitStorage
template class boink::dBG<boink::storage::BitStorage,
                          boink::hashing::RollingHashShifter>;
template class std::enable_shared_from_this<boink::dBG<boink::storage::BitStorage,
                                            boink::hashing::RollingHashShifter>>;
template class boink::AssemblerMixin<boink::dBG<boink::storage::BitStorage,
                                                 boink::hashing::RollingHashShifter>>;
template class boink::CompactorMixin<boink::dBG<boink::storage::BitStorage,
                                                 boink::hashing::RollingHashShifter>>;
template class boink::DecisionNodeProcessor<boink::dBG<boink::storage::BitStorage,
                                                       boink::hashing::RollingHashShifter>>;
template class boink::StreamingCompactorProcessor<boink::dBG<boink::storage::BitStorage,
                                                             boink::hashing::RollingHashShifter>>;
template class